In [3]:
MODEL = "llama3"

from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings()

model.invoke("Tell me sentence with just 5 words")

'"The dog is very happy."'

In [4]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model
chain.invoke("Tell me a joke")

"Here's one:\n\nWhy don't eggs tell jokes?\n\n(wait for it...)\n\nBecause they'd crack each other up!\n\nHope that made you smile!"

In [5]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Champions-league-2023-2data.pdf")
pages = loader.load_and_split()
pages

[Document(page_content='Below are Champions league 2023/24 season all game information:   \n \nGroups information:  \nGroup A: Bayern Munich, Copenhagen, Galatasaray, Manchester United  \nGroup B: Arsenal, PSV, Lens, Sevilla  \nGroup C: Real Madrid, Napoli, Braga, Union Berlin  \nGroup D: Real  Sociedad, Inter, Benfica, RB Salzburg  \nGroup E: Atletico Madrid, Lazio, Feyenoord, Celtic  \nGroup F: Dortmund, PSG, Milan, Newcastle  \nGroup G: Manchester City, RB Leipzig, Young Boys, Red Star Belgrade  \nGroup H: Barcelona, Porto, Shakhtar Donetsk, Antwerp  \n \nGam es results:  \n1. Milan 0 - 0 Newcastle. 19th September  2023, group stage matchday 1 of 6.  \nAC Milan were held to a 0 -0 draw at San Siro in a hard -fought contest by a \nNewcastle United side returning to the Champions League after a two -decade \nabsence as the hosts  spurned numerous opportunities in the Group F opener \non Tuesday.  \nMilan\'s Portugal winger Rafael Leão, who has scored in his last two games, \nwas a con

In [6]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))


Answer the question based on the context below. If you can't
answer the question, reply "I don't know".

Context: Here is some context

Question: Here is a question



In [7]:
chain = prompt | model | parser

In [8]:
chain.invoke(
  {
    "context": "Here is some context",
    "question": "Here is a question"
  }
)

"I'm ready! Please go ahead and provide the context and question. I'll do my best to answer it.\n\n(And if I really don't know the answer, you can be sure I'll let you know!)"

In [9]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(
    pages, 
    embedding=embeddings
)

In [10]:
retriever = vectorstore.as_retriever()

retriever.invoke("Manchester")

[Document(page_content='game. The most important was to attack and create chances. We won the \nmatch thanks to our possession."  \nThere was little  brilliance from the home side in their first grand outing of the \nseason, but PSG were rarely in danger thanks to their domination of the \nmidfield before everything seemed to click after Mbappe coolly buried his \npenalty four minutes into the second half.  \nDortmund had the first clear chance when Donyell Malen found some space in \nthe PSG area, only for his shot to be easily saved by Gianluigi Donnarumma.  \nPSG hit back five minutes later when Vitinha, who bossed the midfield in the \nopening half, hit the woodwo rk with a fine shot from just outside the area.  \nThe Portuguese had another opportunity after half an hour but his attempt \nwas deflected and went over the bar.  \nPSG lacked a cutting edge until early in the second half when Ousmane \nDembélé, after a combinatio n with fullback Hakimi on the right flank, found \nMbapp

In [13]:
from operator import itemgetter

chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | prompt
    | model
    | parser
)

chain.invoke({"question": "Did PSG win Dortmund?"})

"I don't know. The provided context does not mention PSG or Dortmund, and there is no information about a match between the two teams."

In [12]:
questions = [
    "What is the result of Braga and Napoli game?",
    "Did Real Madrid win Union Berlin?",
    "Against who did Bayern Munich play against?",
    "Did PSG and Dortmund play against each other?"
]

for question in questions:
  print(f"Question: {question}")
  print(f"Answer: {chain.invoke({'question': question})}")
  print()

Question: What is the result of Braga and Napoli game?
Answer: According to the context, the result of the Braga vs Napoli game was 1-2 in favor of Napoli.

Question: Did Real Madrid win Union Berlin?
Answer: Yes, according to the context, Real Madrid beat Union Berlin 1-0 at home.

Question: Against who did Bayern Munich play against?
Answer: I don't know. The text does not mention the specific match where Arsenal suffered a "chastening 10 -2 aggregate drubbing at the hands of Bayern Munich", so it is unclear which team they played against in that particular match.

Question: Did PSG and Dortmund play against each other?
Answer: I don't know. There is no information about PSG or Dortmund playing against each other in this context. The documents provided only mention matches involving Arsenal, PSV Eindhoven, Benfica, RB Salzburg, Braga, Napoli, Real Madrid, Union Berlin, Inter, Barcelona, and Antwerp.

